In [108]:
# Processing
This notebook will walk through what the processing is for the 3 different forms, and the trial and error taken to get to those forms.

In [107]:
%pip install numpy matplotlib pandas biopython

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\Cory\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


# Load all neccessary libraries
import numpy as np
import pandas as pd

# Bio libraries needed
from Bio.Align import substitution_matrices

**Load needed variables**

In [109]:
data_dir = "../data/test/"
blosum62 = substitution_matrices.load('BLOSUM62')

# Form 1
The list of annotations are created into a list of scores using the following methods:
- comparing substitutions using BLOSUM-62 matrix
- Frame shifts                    = -10
- Inserts (no matter length)      = -5
- Deletions (no matter length)    = -5
- Duplications (no matter length) = -2

## Getting data
First, we need to get the data. The first column of the CSV file should have Isolate_ID, but this may not be unique as 1 isolate could have many annotations.

In [110]:
ann_df = pd.read_csv(f'{data_dir}export_69.csv')

In [111]:
# See what the original data annotations look like
ann_df.head()

,Isolate_ID,Gene,Type,Ref,Position(s),AA,% coverage
0,1,SHV-12,sub,G,232,R,25
1,2,SHV-12,fs,I,9,NaN,100


## Dropping columns
We do not care about the column \[%], so we can drop that. Even though Isolate_ID is not unique, we still want to include it so we can group by this column later as each datapoint will be a list of annotations using this column.

In [112]:
ann_df = ann_df.drop(['% coverage'], axis=1)
ann_df.head()

,Isolate_ID,Gene,Type,Ref,Position(s),AA
0,1,SHV-12,sub,G,232,R
1,2,SHV-12,fs,I,9,NaN


## Create Score column
Next, we need to create the score column that we will use later.

### Score function
To make it more clear, we can make a score function that will calculate each annotation's score.

In [113]:
def calc_score(row):
    if row.Type == 'sub':
        return blosum62[row.Ref, row.AA]    # Matrix is square, so ref and aa are interchangable
    elif row.Type == 'fs':
        return -10
    elif row.Type == 'ins':
        return -5
    elif row.Type == 'del':
        return -5
    elif row.Type == 'dup':
        return -2

### Apply the score function
Here, the score function is applied over all rows (axis = 1).

In [114]:
ann_df['Score'] = ann_df.apply(calc_score, axis=1)

In [115]:
ann_df.head()

,Isolate_ID,Gene,Type,Ref,Position(s),AA,Score
0,1,SHV-12,sub,G,232,R,-2.0
1,2,SHV-12,fs,I,9,NaN,-10.0


## Drop columns
Now, we do not need Type, Ref, Position(s), and AA.

In [116]:
ann_df = ann_df.drop(['Type', 'Ref', 'Position(s)', 'AA'], axis=1)

In [117]:
ann_df.head()

,Isolate_ID,Gene,Score
0,1,SHV-12,-2.0
1,2,SHV-12,-10.0


## Group all scores into a list per isolate/gene
Our final dataframe needs to have each row being a single datapoint representing an isolate and gene combination. The datapoint's value should be the list of annotations found. We want to go over a few options to see which is more efficient for the amount of data we have.

Code below found in [this StackOverflow answer](https://stackoverflow.com/a/22221675).

In [118]:
# First, we want to groub by id and gene so we get all combined scores
# Second, we want to get the made list of Score rows
# Third, we want to make want to make each row into a list
# Finally, we want to reset the index for each row so that we get individual rows per each element in the group
%timeit ann_df.groupby(['Isolate_ID', 'Gene'])['Score'].apply(list).reset_index(name='ann_lists')

6.84 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This next option is more convienent, but it is twice as slow and has weird output ([found here](https://stackoverflow.com/a/66853746)).

In [119]:
%timeit pd.pivot_table(ann_df, values='Score', index=['Isolate_ID', 'Gene'], aggfunc={'Score': list})

13.2 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Trying the same line as the first one, but this time using `np.array` instead of `list`. It seems to be about the same time as just using `list`.

In [120]:
%timeit ann_df.groupby(['Isolate_ID', 'Gene'])['Score'].apply(np.array).reset_index(name='ann_lists')

6.97 ms ± 98.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Since this will only be run once to generate the datasets, I will use the first option for convienence and ease of reading.

In [121]:
ann_df = ann_df.groupby(['Isolate_ID', 'Gene'])['Score'].apply(list).reset_index(name='ann_lists')

In [122]:
ann_df.head()

,Isolate_ID,Gene,ann_lists
0,1,SHV-12,[-2.0]
1,2,SHV-12,[-10.0]


## To CSV
Lastly, we want to export as CSV without the indecies. We do not need the pandas pre-made indicies, so we can remove them from the CSV file. Since this file is just testing, it will be saved to the current folder.

In [123]:
ann_df.to_csv('form_1.csv', index=False)

Test to see if the file can be loaded, and what it will look like if it is.

In [124]:
pd.read_csv('form_1.csv').head()

,Isolate_ID,Gene,ann_lists
0,1,SHV-12,[-2.0]
1,2,SHV-12,[-10.0]


# Form 2
Create input form 2 where the annotated Amino Acid sequences will be converted into either -1 or 1 given the following criteria:
- -1 if:
    1. The Amino Acid had no mutation
- 1 if:
    1. A SNP occurred at the position
    2. An insertion occurred (no matter how long the inserted sequence was, only a single -1 is given)
    2. A deletion occurred (no matter how long the deleted sequence was, only a single -1 is given)
- Duplications are considered (-1) for the unmuttated Amino Acid and then the rest of the dupplication is treated as a single insert (1).
- Frameshifts are considered a 1 for every position beyond where the frameshift occurred.


## Get and show
First, let's get and see what the data looks like as a dataframe.

In [126]:
msa_df = pd.read_csv(f'{data_dir}export_msa_69.csv')

In [127]:
msa_df.head()

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
0,1,M,R,Y,I,R,L,C,I,I,...,A,A,L,I,E,H,W,Q,R,X
1,2,M,R,Y,I,R,L,C,I,?,...,-,-,-,-,-,-,-,-,-,-
2,consensus,M,R,Y,I,R,L,C,I,I,...,A,A,L,I,E,H,W,Q,R,X
3,reference,M,R,Y,I,R,L,C,I,I,...,A,A,L,I,E,H,W,Q,R,X


## Remove consensus row
We do not need the consensus row, so we can remove that

In [132]:
msa_df = msa_df[msa_df.Name != 'consensus']

In [133]:
msa_df.head()

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
0,1,M,R,Y,I,R,L,C,I,I,...,A,A,L,I,E,H,W,Q,R,X
1,2,M,R,Y,I,R,L,C,I,?,...,-,-,-,-,-,-,-,-,-,-
3,reference,M,R,Y,I,R,L,C,I,I,...,A,A,L,I,E,H,W,Q,R,X


## Change Amino Acids to 1's and -1's
We will next need to remove the reference row and use it to turn the Amino Acid sequence into a sequence of 1's and -1's. The resulting list of numbers should be the same size as the reference.

In [144]:
# Get the reference row, turn it into a list of lists (there will only be 1 row, so 1 list)
# Then, remove the first element which is the name of the row 'reference'
reference = msa_df[msa_df.Name == 'reference'].values.tolist()[0][1:]

# Remove reference row
msa_df = msa_df[msa_df.Name != 'reference']

There needs to be a function that can go over each row, create the list of 1's and -1's, and return that list to be readded to the df.

In [154]:
def create_ones_list(row, reference = []):
    # The first element of the row is the isolate id, so we must remove that first
    row = row[1:]

    # Since both sequnces were part of a MSA, we can assume the lists are the same length.
    frameshift = False
    insert = False
    delete = False
    ones = []
    for p, r in zip(row, reference):

        # Frameshift occurred in the sequence
        if frameshift:
            ones.append(1)
            continue
        elif p == '?':
            frameshift = True
            ones.append(1)
            continue

        # Insert occurred in the sequence meaning reference has a -
        if insert and r == '-':
            continue
        elif r == '-':
            insert = True
            ones.append(1)
            continue
        else:
            insert = False

        # Delete occurred in the sequence meaning sequence has a -
        if delete and p == '-':
            continue
        elif p == '-':
            delete = True
            ones.append(1)
            continue
        else:
            delete = False

        # Either a substitution occurred or the 2 sequences are the same
        if p != r:
            ones.append(1)
        else:
            ones.append(-1)

    return ones

In [159]:
msa_df.head()

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,ones
0,1,M,R,Y,I,R,L,C,I,I,...,A,L,I,E,H,W,Q,R,X,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,2,M,R,Y,I,R,L,C,I,?,...,-,-,-,-,-,-,-,-,-,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1..."


In [160]:

msa_df['ones'] = msa_df.apply(create_ones_list, axis=1, reference=reference)

In [161]:
msa_df.head()

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,ones
0,1,M,R,Y,I,R,L,C,I,I,...,A,L,I,E,H,W,Q,R,X,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,2,M,R,Y,I,R,L,C,I,?,...,-,-,-,-,-,-,-,-,-,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1..."


## Remove all gene columns
Now that we have the ones list created, we can now remove all other gene columns.

In [167]:
msa_df = msa_df.drop(msa_df.columns[[x not in ['Name', 'ones'] for x in msa_df.columns]], axis=1)

In [170]:
msa_df.head()

,Name,ones
0,1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,2,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1..."


## To CSV
Lastly, we want to export as CSV without the indecies. We do not need the pandas pre-made indicies, so we can remove them from the CSV file. Since this file is just testing, it will be saved to the current folder.

In [168]:
msa_df.to_csv('form_2.csv', index=False)

Test to see if the file can be loaded, and what it will look like if it is.

In [169]:
pd.read_csv('form_2.csv').head()

,Name,ones
0,1,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,2,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1..."


# Form 3
Create input form 3 where the annotated Amino Acid sequences will be converted a sequence of numbers corresponding to the character that occurres in the annotated sequence.

NOTE: It does not rely on the reference sequence

## Start
To start, we will do the same things as we did for form 2 up to the reference. We will be removing the reference column and not using it.

In [178]:
msa_df = pd.read_csv(f'{data_dir}export_msa_69.csv', index_col=0)
msa_df = msa_df[msa_df.index != 'consensus']
msa_df = msa_df[msa_df.index != 'reference']

In [179]:
msa_df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,10,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
Name,,,,,,,,,,,,,,,,,,,,,
1,M,R,Y,I,R,L,C,I,I,S,...,A,A,L,I,E,H,W,Q,R,X
2,M,R,Y,I,R,L,C,I,?,-,...,-,-,-,-,-,-,-,-,-,-


## Create function for values
We need to create a conversion function that will take in an Amino Acid (plus a few extra characters) and return an index of that character.

In [187]:
def convert(col):
    characters = {
        'A': 0,
        'R': 1, 
        'N': 2,
        'D': 3,
        'C': 4,
        'Q': 5,
        'E': 6,
        'G': 7,
        'H': 8,
        'I': 9,
        'L': 10,
        'K': 11,
        'M': 12,
        'F': 13,
        'P': 14,
        'S': 15,
        'T': 16,
        'W': 17,
        'Y': 18,
        'V': 19,
        'B': 20,
        'Z': 21,
        'X': 22,
        '?': 23,
        '-': 24
    }

    new_col = []
    for c in col:
        tmp = characters.get(c, None)
        if tmp is None:
            raise ValueError(f'Value {c} does not exist in characters dictionary')

        new_col.append(tmp)

    return new_col

In [181]:
# See old head of dataframe
msa_df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,10,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
Name,,,,,,,,,,,,,,,,,,,,,
1,M,R,Y,I,R,L,C,I,I,S,...,A,A,L,I,E,H,W,Q,R,X
2,M,R,Y,I,R,L,C,I,?,-,...,-,-,-,-,-,-,-,-,-,-


In [189]:
# Apply function
msa_df = msa_df.apply(convert, axis=1, result_type='broadcast')

In [190]:
msa_df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,10,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
Name,,,,,,,,,,,,,,,,,,,,,
1,12,1,18,9,1,10,4,9,9,15,...,0,0,10,9,6,8,17,5,1,22
2,12,1,18,9,1,10,4,9,23,24,...,24,24,24,24,24,24,24,24,24,24


## To CSV
Lastly, we want to export as CSV without the indecies. We do not need the pandas pre-made indicies, so we can remove them from the CSV file. Since this file is just testing, it will be saved to the current folder.

In [193]:
msa_df.to_csv('form_3.csv')

Test to see if the file can be loaded, and what it will look like if it is.

In [194]:
pd.read_csv('form_3.csv').head()

,Name,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 278,Unnamed: 279,280,Unnamed: 281,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287
0,1,12,1,18,9,1,10,4,9,9,...,0,0,10,9,6,8,17,5,1,22
1,2,12,1,18,9,1,10,4,9,23,...,24,24,24,24,24,24,24,24,24,24
